In [3]:
!pip install rouge
!pip install transformers
!pip install tqdm
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import json, gzip
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge import Rouge
import random
from tqdm import tqdm
import nltk
from nltk.translate import meteor_score
import string
nltk.download('wordnet')
nltk.download('wordnet_ic')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


True

In [5]:
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
rouge = Rouge()

In [6]:
path = "/content/test.jsonl.gz"
data = []

with gzip.open(path) as f:
    for ln in f:
        obj = json.loads(ln)
        data.append(obj)


In [10]:
random.shuffle(data)
numArticles = 50
totScore_l = 0
totScore_1 = 0
totScore_2 = 0
metScore = 0
# to remove punctuation
translator = str.maketrans('', '', string.punctuation)

for article in tqdm(data[:numArticles]):
    text = article["text"]
    summary = article["summary"]
    # title = article["title"]

    inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors="pt")
    summary_ids = model.generate(
        inputs["input_ids"], num_beams=4, max_length=200, early_stopping=True
    )
    bart_summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # ROUGE
    scores = rouge.get_scores(bart_summary, summary)
    totScore_l += scores[0]["rouge-l"]["f"]
    totScore_1 += scores[0]["rouge-1"]["f"]
    totScore_2 += scores[0]["rouge-2"]["f"]

    # METEOR
    ref = bart_summary.translate(translator).split()
    hyp = summary.translate(translator).split()
    score = meteor_score.meteor_score([ref], hyp)
    metScore += score

print()
print("Rouge_l")
print("Total: " + str(totScore_l))
print("Average: " + str(totScore_l / numArticles))
print()
print("Rouge_1")
print("Total: " + str(totScore_1))
print("Average: " + str(totScore_1 / numArticles))
print()
print("Rouge_2")
print("Total: " + str(totScore_2))
print("Average: " + str(totScore_2 / numArticles))
print()
print("METEOR")
print("Total: " + str(metScore))
print("Average: " + str(metScore / numArticles))

100%|██████████| 50/50 [11:14<00:00, 13.48s/it]


Rouge_l
Total: 11.214123401590479
Average: 0.22428246803180957

Rouge_1
Total: 12.007073639675331
Average: 0.24014147279350662

Rouge_2
Total: 5.3082576076762615
Average: 0.10616515215352523

METEOR
Total: 8.15201727894564
Average: 0.1630403455789128
